## Imports

In [128]:
import pandas as pd
from neo4j import GraphDatabase

## Connection

In [129]:
URI      = "neo4j://localhost:7687"
creds    = ('neo4j', 'password')
driver   = GraphDatabase.driver(URI, auth=creds)

drop = True

In [130]:
def run_cypher(cypher, results=False):
    with driver.session() as session:
        r = session.run(cypher).data()
    if results:
        return r

## Dataset
[Human protein-protein interaction network](https://snap.stanford.edu/biodata/datasets/10000/10000-PP-Pathways.html)

In [131]:
pd.read_csv('https://raw.githubusercontent.com/seankrobinson/Protein-Interaction_Link-Prediction/master/PP-Pathways_ppi.csv',
            header=None).head()

,0,1
0,1394,2778
1,6331,17999
2,122704,54460
3,2597,2911
4,4790,79155


## Import Dataset

In [132]:
c = '''
// Create constraints
CREATE CONSTRAINT proteins IF NOT EXISTS ON (p:Protein) ASSERT p.id IS UNIQUE;
'''
run_cypher(c)

## Create Nodes

In [133]:
for i in [0,1]:
    c = '''
    // Load node list
    LOAD CSV FROM 'https://raw.githubusercontent.com/seankrobinson/Protein-Interaction_Link-Prediction/master/PP-Pathways_ppi.csv' AS line
    WITH line
    MERGE (p:Protein {{id: line[{index}]}})
    RETURN COUNT(DISTINCT p)
    '''
    result = run_cypher(c.format(index=i), results=True)
    print(result)

[{'COUNT(DISTINCT p)': 15849}]
[{'COUNT(DISTINCT p)': 19341}]


## Create Edges

In [134]:
c = '''
// Load node list
LOAD CSV FROM 'https://raw.githubusercontent.com/seankrobinson/Protein-Interaction_Link-Prediction/master/PP-Pathways_ppi.csv' AS line
WITH line
MATCH (p1:Protein {id: line[0]})
WITH line, p1
MATCH (p2:Protein {id: line[1]})
MERGE (p1)-[r:INTERACTS_WITH]->(p2)
RETURN COUNT(r)
'''
result = run_cypher(c, results=True)
print(result)

[{'COUNT(r)': 342353}]


## Create Link Prediction Pipeline

In [ ]:
run_cypher("CALL gds.beta.model.drop('pipe')", results=True) if drop else ''

c = f'''
CALL gds.alpha.ml.pipeline.linkPrediction.create('pipe')
'''
print(c)
run_cypher(c, results=True)

## Add Fast RP Embeddings

In [ ]:
c = '''
CALL gds.alpha.ml.pipeline.linkPrediction.addNodeProperty('pipe', 'fastRP', {
  mutateProperty: 'embedding',
  embeddingDimension: 256,
  iterationWeights: [0.8, 1, 1, 1],
  normalizationStrength: 0.5,
  randomSeed: 42
})
'''
print(c)
run_cypher(c, results=True)

## Add pageRank Feature

In [ ]:
c = '''
CALL gds.alpha.ml.pipeline.linkPrediction.addNodeProperty('pipe', 'pageRank', 
{
  mutateProperty: 'pageRank'
})
'''
print(c)
run_cypher(c, results=True)

## Add Betweenness Centrality Feature

In [ ]:
c = '''
CALL gds.alpha.ml.pipeline.linkPrediction.addNodeProperty('pipe', 'betweenness', 
{
  mutateProperty: 'betweenness'
})
'''
print(c)
run_cypher(c, results=True)

In [ ]:
c = '''
CALL gds.alpha.ml.pipeline.linkPrediction.addFeature('pipe', 'hadamard', {
  nodeProperties: ['embedding']
}) YIELD featureSteps
'''
print(c)
run_cypher(c, results=True)

In [ ]:
c = '''
CALL gds.alpha.ml.pipeline.linkPrediction.configureSplit('pipe', {
  testFraction: 0.3,
  trainFraction: 0.3,
  //negativeSamplingRatio: 1.33,
  validationFolds: 7
  
})
YIELD splitConfig
'''
print(c)
run_cypher(c, results=True)

## Confifure Model Params

In [ ]:
c = '''
CALL gds.alpha.ml.pipeline.linkPrediction.configureParams('pipe',
  [{penalty:0, tolerance: 0.001, maxEpochs: 500},
   {penalty:0, tolerance: 0.01, maxEpochs: 500},
   {penalty:0.01, tolerance: 0.001, maxEpochs: 500},
   {penalty:0.01, tolerance: 0.01, maxEpochs: 500}, 
   {penalty:0.1, tolerance: 0.001, maxEpochs: 500}, 
   {penalty:0.1, tolerance: 0.01, maxEpochs: 500} ]
) YIELD parameterSpace
'''
print(c)
run_cypher(c, results=True)

## Create graph projection

In [ ]:
run_cypher('call gds.graph.drop("proteins")') if drop else ''

c = '''
// Create in-memory graph of (Protein)-[:INTERACTS_WITH]-(Protein)
CALL gds.graph.create(
    'proteins',
    'Protein',
    {INTERACTS_WITH: 
        {
            orientation: 'UNDIRECTED'
        }
    }
)
'''
run_cypher(c, results=True)

## Train link prediction model

In [ ]:
run_cypher("call gds.beta.model.drop('lp-pipeline-model')", results=True) if drop else ''

c = '''
CALL gds.alpha.ml.pipeline.linkPrediction.train('proteins', {
  pipeline: 'pipe',
  modelName: 'lp-pipeline-model',
  randomSeed: 42
}) YIELD modelInfo
RETURN
  modelInfo.bestParameters AS winningModel,
  modelInfo.metrics.AUCPR.outerTrain AS trainGraphScore,
  modelInfo.metrics.AUCPR.test AS testGraphScore
'''
print(c)
run_cypher(c, results=True)

## Stream Results

In [ ]:
c = '''
CALL gds.alpha.ml.pipeline.linkPrediction.predict.stream('proteins', {
  modelName: 'lp-pipeline-model',
  topN: 20,
  threshold: 0.3
})
 YIELD node1, node2, probability


 RETURN gds.util.asNode(node1).id AS protein1, gds.util.asNode(node2).id AS protein2, probability
 ORDER BY probability DESC, protein1
'''
print(c)
results = run_cypher(c, results=True)

In [ ]:
pd.DataFrame(results).head()